In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key=''

# anthropic API key
claude_api_key = ''

# together AI API key
together_api_key = ''

In [2]:
# read the prompt from the prompt text file titled "ironore.txt"
with open("../prompts/ironore.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)

In [3]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT-4o

In [7]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [8]:
print(response)

To solve this problem, we can approach it using a simulation-based method due to the complexity of the stochastic and threshold-based decisions involved. Here's how we can tackle each part:

### Method Explanation

1. **Price Process**: We model the daily price \( P_t \) as a mean-reverting random walk. The mean reversion suggests that prices will tend to drift back towards the mean price \( \mu_0 \) over time. We truncate the price between `min_price` and `max_price` to avoid extreme values.

2. **Production and Inventory Management**: 
   - Threshold Logic:
     - **Starting Production (x_1)**: When the price exceeds \( x_1 \) and inventory is below \( x_2 \).
     - **Stopping Production (x_3 and x_2)**: Stop if price is below \( x_3 \) or inventory is above \( x_2 \).
     - **Selling All Stock (x_4)**: If the price exceeds \( x_4 \), all inventory is sold.

3. **Costs and Revenue**:
   - Production incurs a cost per unit \( c \) when units are produced.
   - Holding cost \( h \) i

In [9]:
import numpy as np

# Function to simulate price generation
def generate_price_series(T, mean_price, sigma, min_price, max_price):
    prices = np.zeros(T)
    prices[0] = mean_price
    for t in range(1, T):
        drift = (mean_price - prices[t - 1]) ** (1/4)
        shock = sigma * np.random.randn()
        prices[t] = np.clip(prices[t - 1] + drift + shock, min_price, max_price)
    return prices

# Function to simulate production and sales
def simulate_production(T, prices, params):
    x_1, x_2, x_3, x_4 = params
    inventory = 0
    total_profit = 0
    frac_producing_days = 0

    production = False
    for t in range(T):
        P_t = prices[t]
        if production:
            if P_t <= x_3 or inventory >= x_2:
                production = False
            else:
                produce_amount = min(100, 10000 - inventory)
                inventory += produce_amount
                total_profit -= produce_amount * 100
                
        if not production and P_t >= x_1 and inventory < x_2:
            production = True
        
        if P_t >= x_4:
            total_profit += inventory * P_t
            inventory = 0

        total_profit -= inventory * 1
        
        frac_producing_days += int(production)
    
    mean_stock = inventory / T
    frac_producing = frac_producing_days / T
    return total_profit, frac_producing, mean_stock

# Parameter Searching
def optimize_thresholds(budget, T, mean_price, sigma, min_price, max_price):
    best_profit = -np.inf
    best_params = None
    np.random.seed(0)  # For reproducibility

    for _ in range(budget):
        x_1 = np.random.uniform(0, max_price)
        x_2 = np.random.uniform(0, 10000)
        x_3 = np.random.uniform(0, x_1)
        x_4 = np.random.uniform(x_1, max_price)

        prices = generate_price_series(T, mean_price, sigma, min_price, max_price)
        total_profit, frac_producing, mean_stock = simulate_production(T, prices, [x_1, x_2, x_3, x_4])

        if total_profit > best_profit:
            best_profit = total_profit
            best_params = (x_1, x_2, x_3, x_4)

    return best_params, best_profit

# Define Constants
T = 365
mean_price = 100.0
max_price = 200.0
min_price = 0.0
sigma = 7.5

# Optimization
best_params, best_profit = optimize_thresholds(budget=1000, T=T, mean_price=mean_price, sigma=sigma, min_price=min_price, max_price=max_price)

# Output Results
best_params, best_profit

/var/folders/8q/jm2k0n2915zgdz45fqf7jjvc0000gr/T/ipykernel_17909/2722687112.py:8: RuntimeWarning: invalid value encountered in scalar power
  drift = (mean_price - prices[t - 1]) ** (1/4)


((16.900569711004977,
  153.34646908561012,
  4.114568809189722,
  102.03426560785628),
 np.float64(1222.7393717288178))

## o1

In [10]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [11]:
print(response)

Below is an outline of how one might tackle the problem in both an analytical/heuristic way (Part A) and via simulation-based search (Part B). In practice, many real-world implementations would default to a simulation-based search, since a truly closed-form solution to this threshold-based policy (under a truncated, mean‐reverting price) is quite challenging. Nonetheless, we can propose approximate arguments to guide threshold selection, then refine numerically.

────────────────────────────────────────────────────────────────────────────
A) ANALYTICAL / HEURISTIC APPROACH
────────────────────────────────────────────────────────────────────────────
1) Price Model Approximation
   Let Pₜ follow a mean-reverting process around mean_price = μ₀, truncated to [min_price, max_price]. Symbolically, one might write:
     Pₜ₊₁ = Pₜ + α·(|μ₀ - Pₜ|)^(1/4)·sgn(μ₀ - Pₜ) + σ·εₜ
   where εₜ ~ N(0,1). Then clamp Pₜ₊₁ to [min_price, max_price].  

   For a rough approximation, you might consider a simp

In [12]:
import numpy as np

def generate_prices(n_days=365,
                    mean_price=100.0,
                    min_price=0.0,
                    max_price=200.0,
                    sigma=7.5,
                    alpha=0.5,
                    seed=None):
    """
    Generate a daily price path via a truncated mean-reverting random walk.
    P_{t+1} = P_t + alpha * ( (mu_0 - P_t)**(1/4) ) * sign(mu_0 - P_t ) + sigma * Z_t
    Then clamp to [min_price, max_price].
    """
    if seed is not None:
        np.random.seed(seed)
        
    prices = np.zeros(n_days)
    # Initialize price near the mean, or arbitrarily
    P = mean_price
    for t in range(n_days):
        # drift term
        drift = alpha * (abs(mean_price - P)**0.25)
        if P > mean_price:
            drift = -drift
        # random shock
        shock = sigma * np.random.randn()
        # next price
        P_next = P + drift + shock
        # clamp
        P_next = np.clip(P_next, min_price, max_price)
        prices[t] = P_next
        P = P_next
    return prices

def simulate_policy(x1, x2, x3, x4,
                    prices,
                    prod_cost=100.0,
                    holding_cost=1.0,
                    max_prod_perday=100,
                    capacity=10000,
                    verbose=False):
    """
    Simulate the threshold policy over the given price path.
    Returns total_profit, frac_producing, mean_stock.
    """
    n_days = len(prices)
    stock = 0
    profit = 0.0
    
    # Keep track of production status
    producing = False
    producing_days = 0
    
    for t, P_t in enumerate(prices):
        # (a) Observe today's price (P_t) and current stock (stock).
        
        # (b) If currently producing:
        if producing:
            # Check stop conditions:
            if (P_t <= x3) or (stock >= x2):
                producing = False
            else:
                # Produce up to max_prod_perday or capacity minus current stock
                prod_units = min(max_prod_perday, capacity - stock)
                if prod_units > 0:
                    cost_today = prod_units * prod_cost
                    profit -= cost_today
                    stock += prod_units
                producing_days += 1
        
        # (c) If NOT producing, check if we should start
        else:
            if (P_t >= x1) and (stock < x2):
                producing = True
                # produce immediately on this day
                prod_units = min(max_prod_perday, capacity - stock)
                if prod_units > 0:
                    cost_today = prod_units * prod_cost
                    profit -= cost_today
                    stock += prod_units
                producing_days += 1
        
        # (d) If price >= x4, sell all stock
        if P_t >= x4 and stock > 0:
            revenue = stock * P_t
            profit += revenue
            stock = 0
        
        # (e) Pay holding cost for remaining stock
        if stock > 0:
            profit -= (stock * holding_cost)
    
    frac_producing = producing_days / n_days
    mean_stock = 0  # We can do a more sophisticated average, see note below
    # For a simple approach, add up daily stock. This requires a minor code change above.
    
    return profit, frac_producing, mean_stock

def random_search_optimization(budget=1000,
                               n_days=365,
                               mean_price=100.0,
                               min_price=0.0,
                               max_price=200.0,
                               sigma=7.5,
                               alpha=0.5,
                               prod_cost=100.0,
                               holding_cost=1.0,
                               max_prod_perday=100,
                               capacity=10000,
                               seed=1234):
    """
    Perform a random search of the threshold space within reasonable bounds.
    x1, x2, x3, x4 each in [0, some_max].
    Returns the best found thresholds and associated profit.
    """
    np.random.seed(seed)
    
    # Generate a single price path for fairness, or multiple if you want repeated averaging
    prices = generate_prices(n_days=n_days, mean_price=mean_price,
                             min_price=min_price, max_price=max_price,
                             sigma=sigma, alpha=alpha, seed=seed)
    
    best_profit = -1e15
    best_x1x2x3x4 = (0,0,0,0)
    
    for _ in range(budget):
        # Sample thresholds from a range
        x1 = np.random.uniform(0, 2*mean_price)  # e.g. up to double mean
        x3 = np.random.uniform(0, 2*mean_price)
        x2 = np.random.uniform(0, capacity)      # up to capacity
        x4 = np.random.uniform(0, 2*mean_price)
        
        # Possibly impose x3 <= x1 <= x4 by sorting
        # But the question does not strictly enforce it, so we skip or do partial:
        if x3 > x1:
            x3, x1 = x1, x3
        if x4 < x1:
            x4, x1 = x1, x4
        
        profit, frac_prod, mean_stk = simulate_policy(
            x1, x2, x3, x4,
            prices=prices,
            prod_cost=prod_cost,
            holding_cost=holding_cost,
            max_prod_perday=max_prod_perday,
            capacity=capacity,
            verbose=False
        )
        
        if profit > best_profit:
            best_profit = profit
            best_x1x2x3x4 = (x1, x2, x3, x4)
    
    return best_x1x2x3x4, best_profit

if __name__ == "__main__":
    best_thresholds, best_profit = random_search_optimization(budget=1000)
    print("Best thresholds found:", best_thresholds)
    print("Best profit estimate:", best_profit)

Best thresholds found: (90.4303092157075, 8355.62083918515, 40.149614024404755, 109.93694856488638)
Best profit estimate: 532537.4474603242


## o3-mini

In [19]:
completion = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [20]:
print(response)

Below is one complete answer that explains our methodology, provides a full Python‐code simulation (all in one fenced block), and then reports “results” comparing our tuned thresholds versus a baseline.

────────────────────────────
1) Method Explanation
────────────────────────────
Because a closed‐form solution is intractable under the nonlinear threshold rules and the truncated mean‐reverting price process, we use a simulation‐based optimization. The idea is as follows:

• Price process:
  – We simulate the daily spot price Pₜ as a mean‐reverting random walk with bounds.
  – In our simulation, the “drift” is given by 
       drift = sign(mean_price – Pₜ₋₁) × |mean_price – Pₜ₋₁|^(1/4)
    so that when prices are far from mean_price the “pull” is stronger (albeit raised to the 1/4 power).
  – A normal noise term (with standard deviation sigma) is added, and we truncate the resulting price to lie in [min_price, max_price].

• Threshold logic:
  – At the start of each day we check wheth

In [21]:
import numpy as np

# Set default parameters for the model
params = {
    "mean_price": 100.0,
    "max_price": 200.0,
    "min_price": 0.0,
    "capacity": 10000,
    "sigma": 7.5,
    "holding_cost": 1.0,
    "prod_cost": 100.0,
    "max_prod_perday": 100,
    "n_days": 365
}

def simulate_one_run(x1, x2, x3, x4, params, seed=None):
    """
    Simulate one run of the mine over T days with a threshold policy.
    
    Thresholds:
      x1: Production start threshold (price_prod)
      x2: Inventory threshold above which we cease producing (inven_stop)
      x3: Price threshold below which production stops (price_stop)
      x4: Price threshold for selling (price_sell)
    
    Returns:
      total_profit, days_producing, inventory_levels (list)
    """
    if seed is not None:
        np.random.seed(seed)
        
    T = params["n_days"]
    capacity = params["capacity"]
    sigma = params["sigma"]
    mean_price = params["mean_price"]
    max_price = params["max_price"]
    min_price = params["min_price"]
    prod_cost = params["prod_cost"]
    max_prod = params["max_prod_perday"]
    holding_cost = params["holding_cost"]
    
    # Start price at mean
    price = mean_price
    inventory = 0.0
    total_profit = 0.0
    producing = False
    days_producing = 0
    inventory_history = []
    
    for t in range(T):
        # Generate today's price using mean-reverting random walk
        # Use drift = sign(mean_price-P) * abs(mean_price-P)^(1/4)
        drift = np.sign(mean_price - price) * (abs(mean_price - price))**0.25
        noise = sigma * np.random.randn()
        new_price = price + drift + noise
        # Truncate to [min_price, max_price]
        new_price = max(min(new_price, max_price), min_price)
        price = new_price

        # Decision Step 1: Production decision.
        # If already producing: stop if price <= x3 or inventory >= x2, else produce.
        if producing:
            if (price <= x3) or (inventory >= x2):
                producing = False
            else:
                # produce production = min(max_prod, capacity - current inventory)
                qty = min(max_prod, capacity - inventory)
                cost = qty * prod_cost
                total_profit -= cost
                inventory += qty
                days_producing += 1
        # If not producing, then we may start production if price >= x1 and inventory < x2.
        if not producing and (price >= x1) and (inventory < x2):
            producing = True
            # Immediately produce on this day if allowed:
            qty = min(max_prod, capacity - inventory)
            cost = qty * prod_cost
            total_profit -= cost
            inventory += qty
            days_producing += 1

        # Decision Step 2: Sales decision.
        if price >= x4:
            # sell all inventory 
            revenue = inventory * price
            total_profit += revenue
            inventory = 0.0  # clear inventory once sold

        # Holding cost is applied on remaining inventory at end of day.
        total_profit -= inventory * holding_cost
        
        inventory_history.append(inventory)

    return total_profit, days_producing, inventory_history

def simulate_strategy(x1, x2, x3, x4, params, n_sim=50):
    """
    Run several simulation replications for given thresholds.
    Returns the mean profit, production fraction (fraction of days with production)
    and mean inventory along with std error for profit.
    """
    profits = []
    prod_days = []
    inv_end = []
    for s in range(n_sim):
        seed = s  # Different seed for each replication
        profit, days_prod, inv_history = simulate_one_run(x1, x2, x3, x4, params, seed)
        profits.append(profit)
        prod_days.append(days_prod)
        inv_end.append(np.mean(inv_history))
    mean_profit = np.mean(profits)
    profit_std = np.std(profits, ddof=1)
    conf_interval = 1.96 * profit_std / np.sqrt(n_sim)
    mean_frac_prod = np.mean(prod_days) / params["n_days"]
    mean_inventory = np.mean(inv_end)
    return mean_profit, conf_interval, mean_frac_prod, mean_inventory

# --- Optimization by random search ---
def random_search_optimization(params, budget=1000, n_sim=20):
    """
    Randomly search over candidate thresholds.
    Candidate ranges (tuned based on problem context):
      x1: production start threshold in [60, 120]
      x2: inventory stop in [5000, capacity] 
      x3: production stop price in [10, 80]
      x4: selling threshold in [80, 220]
    Returns the best found threshold set and its performance.
    """
    best_profit = -np.inf
    best_candidate = None
    best_stats = None
    
    for i in range(budget):
        cand_x1 = np.random.uniform(60, 120)
        cand_x2 = np.random.uniform(5000, params["capacity"])
        cand_x3 = np.random.uniform(10, 80)
        cand_x4 = np.random.uniform(80, 220)
        
        mean_profit, conf_int, mean_frac_prod, mean_inventory = simulate_strategy(
            cand_x1, cand_x2, cand_x3, cand_x4, params, n_sim=n_sim)
        
        if mean_profit > best_profit:
            best_profit = mean_profit
            best_candidate = (cand_x1, cand_x2, cand_x3, cand_x4)
            best_stats = (mean_profit, conf_int, mean_frac_prod, mean_inventory)
    
    return best_candidate, best_stats

# --- Main Execution ---

# 1. Evaluate baseline policy:
baseline = (80.0, 7000.0, 40.0, 100.0)
baseline_profit, baseline_conf, baseline_frac_prod, baseline_mean_inv = simulate_strategy(
    baseline[0], baseline[1], baseline[2], baseline[3], params, n_sim=50)
print("Baseline Policy (x1, x2, x3, x4):", baseline)
print("Baseline Profit: {:.2f} ± {:.2f}".format(baseline_profit, baseline_conf))
print("Baseline Fraction Producing: {:.3f}, Mean Inventory: {:.2f}".format(baseline_frac_prod, baseline_mean_inv))

# 2. Optimization (random search) within evaluation budget:
best_candidate, best_stats = random_search_optimization(params, budget=1000, n_sim=20)
print("\nOptimized Policy (x1, x2, x3, x4):", best_candidate)
print("Optimized Profit: {:.2f} ± {:.2f}".format(best_stats[0], best_stats[1]))
print("Optimized Fraction Producing: {:.3f}, Mean Inventory: {:.2f}".format(best_stats[2], best_stats[3]))

Baseline Policy (x1, x2, x3, x4): (80.0, 7000.0, 40.0, 100.0)
Baseline Profit: 124070.18 ± 38407.40
Baseline Fraction Producing: 0.993, Mean Inventory: 400.83

Optimized Policy (x1, x2, x3, x4): (119.1540389971267, 8585.981143439758, 52.957132332954714, 158.82874019466098)
Optimized Profit: -3056202.02 ± 289246.25
Optimized Fraction Producing: 0.256, Mean Inventory: 6265.53


## Claude 3.5 Sonnet

In [16]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll tackle this problem using a combination of analytical insights and simulation-based optimization. Let me break down the approach:

1. ANALYTICAL INSIGHTS
The problem has characteristics of an inventory control system with stochastic demand/prices. Some key observations:

- Production should start when price is high enough to cover production cost plus expected holding cost
- The selling threshold x_4 should be above production cost plus some margin to ensure profit
- There's a natural ordering: x_3 < x_1 < x_4
- Inventory stop level x_2 should balance storage costs against opportunity costs

2. SIMULATION-BASED APPROACH
I'll implement a simulation with parameter tuning. Here's the code:

```python
import numpy as np
from scipy.optimize import differential_evolution

def simulate_price_process(T, mean_price, sigma, min_price, max_price, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    prices = np.zeros(T)
    prices[0] = mean_price
    
    for t in range(

In [18]:
import numpy as np
from scipy.optimize import differential_evolution

def simulate_price_process(T, mean_price, sigma, min_price, max_price, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    prices = np.zeros(T)
    prices[0] = mean_price
    
    for t in range(1, T):
        drift = ((mean_price - prices[t-1]) ** 0.25)
        step = drift + sigma * np.random.randn()
        prices[t] = np.clip(prices[t-1] + step, min_price, max_price)
    
    return prices

def run_simulation(params, fixed_params, n_days, seed=None):
    # Unpack parameters
    price_prod, inven_stop, price_stop, price_sell = params
    mean_price = fixed_params['mean_price']
    max_price = fixed_params['max_price']
    min_price = fixed_params['min_price']
    capacity = fixed_params['capacity']
    sigma = fixed_params['st_dev']
    holding_cost = fixed_params['holding_cost']
    prod_cost = fixed_params['prod_cost']
    max_prod_perday = fixed_params['max_prod_perday']
    
    # Generate prices
    prices = simulate_price_process(n_days, mean_price, sigma, min_price, max_price, seed)
    
    # Initialize tracking variables
    inventory = 0
    total_profit = 0
    is_producing = False
    days_producing = 0
    
    for t in range(n_days):
        current_price = prices[t]
        
        # Production decision
        if is_producing:
            if current_price <= price_stop or inventory >= inven_stop:
                is_producing = False
            else:
                production = min(max_prod_perday, capacity - inventory)
                inventory += production
                total_profit -= production * prod_cost
                days_producing += 1
        else:
            if current_price >= price_prod and inventory < inven_stop:
                is_producing = True
                production = min(max_prod_perday, capacity - inventory)
                inventory += production
                total_profit -= production * prod_cost
                days_producing += 1
        
        # Selling decision
        if current_price >= price_sell:
            total_profit += inventory * current_price
            inventory = 0
        
        # Holding cost
        total_profit -= inventory * holding_cost
    
    return -total_profit  # Return negative profit for minimization

def optimize_thresholds(fixed_params, n_days, budget):
    bounds = [
        (0, fixed_params['max_price']),  # price_prod
        (0, fixed_params['capacity']),   # inven_stop
        (0, fixed_params['max_price']),  # price_stop
        (0, fixed_params['max_price'])   # price_sell
    ]
    
    def objective(params):
        trials = 5  # Multiple trials for robustness
        return np.mean([run_simulation(params, fixed_params, n_days, seed=i) 
                       for i in range(trials)])
    
    result = differential_evolution(
        objective, 
        bounds,
        maxiter=budget//5,  # Adjust based on budget
        seed=42
    )
    
    return result.x

# Example usage
fixed_params = {
    'mean_price': 100.0,
    'max_price': 200.0,
    'min_price': 0.0,
    'capacity': 10000,
    'st_dev': 7.5,
    'holding_cost': 1.0,
    'prod_cost': 100.0,
    'max_prod_perday': 100
}

# Optimize thresholds
n_days = 365
budget = 1000
optimal_thresholds = optimize_thresholds(fixed_params, n_days, budget)

# Validate results
n_validation_runs = 50
validation_profits = [-run_simulation(optimal_thresholds, fixed_params, n_days, seed=i) 
                     for i in range(n_validation_runs)]

print(f"Optimal thresholds:")
print(f"price_prod (x_1): {optimal_thresholds[0]:.2f}")
print(f"inven_stop (x_2): {optimal_thresholds[1]:.2f}")
print(f"price_stop (x_3): {optimal_thresholds[2]:.2f}")
print(f"price_sell (x_4): {optimal_thresholds[3]:.2f}")
print(f"\nMean profit: {np.mean(validation_profits):.2f}")
print(f"Std profit: {np.std(validation_profits):.2f}")

/var/folders/8q/jm2k0n2915zgdz45fqf7jjvc0000gr/T/ipykernel_17909/1161416990.py:12: RuntimeWarning: invalid value encountered in scalar power
  drift = ((mean_price - prices[t-1]) ** 0.25)


Optimal thresholds:
price_prod (x_1): 133.57
inven_stop (x_2): 9679.69
price_stop (x_3): 135.89
price_sell (x_4): 167.05

Mean profit: 0.00
Std profit: 0.00


## DeepSeek-R1

In [22]:
import numpy as np

def simulate(params, T, seed=None):
    x1 = params['x1']
    x2 = params['x2']
    x3 = params['x3']
    x4 = params['x4']
    mean_price = params['mean_price']
    max_price = params['max_price']
    min_price = params['min_price']
    capacity = params['capacity']
    sigma = params['sigma']
    h = params['h']
    c = params['c']
    max_prod = params['max_prod_perday']
    
    np.random.seed(seed)
    stock = 0.0
    is_producing = False
    total_profit = 0.0
    days_producing = 0
    total_stock = 0.0
    P = mean_price
    
    for _ in range(T):
        produced = 0
        production_cost = 0.0
        if is_producing:
            if P <= x3 or stock >= x2:
                is_producing = False
            else:
                possible_production = min(max_prod, capacity - stock)
                produced = possible_production
                production_cost = c * produced
                stock += produced
        else:
            if P >= x1 and stock < x2:
                is_producing = True
                possible_production = min(max_prod, capacity - stock)
                produced = possible_production
                production_cost = c * produced
                stock += produced
        
        if is_producing:
            days_producing += 1
        
        if P >= x4:
            revenue = P * stock
            stock = 0.0
        else:
            revenue = 0.0
        
        holding_cost_today = h * stock
        total_profit += revenue - production_cost - holding_cost_today
        total_stock += stock
        
        drift = np.sign(mean_price - P) * (np.abs(mean_price - P) ** 0.25)
        P_next = P + drift + sigma * np.random.normal()
        P_next = np.clip(P_next, min_price, max_price)
        P = P_next
    
    return total_profit, days_producing / T, total_stock / T

def optimize_thresholds(params_default, budget=1000, n_replications=5):
    best_profit = -np.inf
    best_params = None
    
    for _ in range(budget):
        x1 = np.random.uniform(params_default['min_price'], params_default['max_price'])
        x2 = np.random.uniform(0, params_default['capacity'])
        x3 = np.random.uniform(params_default['min_price'], params_default['max_price'])
        x4 = np.random.uniform(params_default['min_price'], params_default['max_price'])
        
        candidate_params = {
            'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4,
            'mean_price': params_default['mean_price'],
            'max_price': params_default['max_price'],
            'min_price': params_default['min_price'],
            'capacity': params_default['capacity'],
            'sigma': params_default['sigma'],
            'h': params_default['h'],
            'c': params_default['c'],
            'max_prod_perday': params_default['max_prod_perday']
        }
        
        profits = []
        for seed in range(n_replications):
            profit, _, _ = simulate(candidate_params, params_default['n_days'], seed=seed)
            profits.append(profit)
        avg_profit = np.mean(profits)
        
        if avg_profit > best_profit:
            best_profit = avg_profit
            best_params = candidate_params.copy()
            best_params['avg_profit'] = avg_profit
            best_params['std_profit'] = np.std(profits)
    
    return best_params

params_default = {
    'mean_price': 100.0,
    'max_price': 200.0,
    'min_price': 0.0,
    'capacity': 10000,
    'sigma': 7.5,
    'h': 1.0,
    'c': 100.0,
    'max_prod_perday': 100,
    'n_days': 365
}

best_params = optimize_thresholds(params_default, budget=1000, n_replications=5)

n_eval = 100
profits_eval = []
for seed in range(n_eval):
    profit, _, _ = simulate(best_params, params_default['n_days'], seed=seed)
    profits_eval.append(profit)
mean_profit = np.mean(profits_eval)
std_profit = np.std(profits_eval)
conf_interval = (mean_profit - 1.96 * std_profit / np.sqrt(n_eval), mean_profit + 1.96 * std_profit / np.sqrt(n_eval))

baseline_params = {
    'x1': 80.0, 'x2': 7000.0, 'x3': 40.0, 'x4': 100.0,
    'mean_price': 100.0,
    'max_price': 200.0,
    'min_price': 0.0,
    'capacity': 10000,
    'sigma': 7.5,
    'h': 1.0,
    'c': 100.0,
    'max_prod_perday': 100
}

profits_baseline = []
for seed in range(n_eval):
    profit, _, _ = simulate(baseline_params, 365, seed=seed)
    profits_baseline.append(profit)
mean_baseline = np.mean(profits_baseline)
std_baseline = np.std(profits_baseline)
conf_baseline = (mean_baseline - 1.96 * std_baseline / np.sqrt(n_eval), mean_baseline + 1.96 * std_baseline / np.sqrt(n_eval))

print("Optimized Parameters:")
print(f"x1: {best_params['x1']:.2f}, x2: {best_params['x2']:.2f}, x3: {best_params['x3']:.2f}, x4: {best_params['x4']:.2f}")
print(f"Average Profit: {mean_profit:.2f} ± {1.96 * std_profit / np.sqrt(n_eval):.2f}")
print(f"95% CI: [{conf_interval[0]:.2f}, {conf_interval[1]:.2f}]")
print("\nBaseline Parameters:")
print(f"x1: 80.00, x2: 7000.00, x3: 40.00, x4: 100.00")
print(f"Average Profit: {mean_baseline:.2f} ± {1.96 * std_baseline / np.sqrt(n_eval):.2f}")
print(f"95% CI: [{conf_baseline[0]:.2f}, {conf_baseline[1]:.2f}]")
print(f"Improvement: {mean_profit - mean_baseline:.2f} ({(mean_profit / mean_baseline - 1) * 100:.2f}%)")

Optimized Parameters:
x1: 38.88, x2: 3043.42, x3: 151.48, x4: 92.14
Average Profit: 52339.77 ± 14054.01
95% CI: [38285.76, 66393.78]

Baseline Parameters:
x1: 80.00, x2: 7000.00, x3: 40.00, x4: 100.00
Average Profit: 140193.38 ± 28359.55
95% CI: [111833.83, 168552.93]
Improvement: -87853.61 (-62.67%)
